In [17]:
import eeconvert as eeconvert
import ee
import geemap
import geopandas as gpd
import rasterio as rio
import numpy as np
import rasterio.features as features
from pathlib import Path
import os
import shutil
from shapely.geometry import Point
import numpy as np
import time

ee.Initialize()


class DatesHelper:
    def __init__(self, DATA_DIR, AOI, DATE_RANGE):
        self.data_dir = DATA_DIR
        self.aoi = AOI
        self.date_range = DATE_RANGE

    def extract_best_dates(self):
        start = time.time()
        # Get best date for each tile
        aoi = eeconvert.gdfToFc(gpd.read_file(self.aoi))
        afghanistan = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0").filter("ADM0_NAME == 'Afghanistan'");
        modis = ee.ImageCollection('MODIS/061/MOD13Q1').filter(ee.Filter.date(self.date_range[0], self.date_range[1]));
        dates = modis.map(lambda x: ee.Feature(None, {'date': x.date().format('YYYY-MM-dd')})).distinct('date').aggregate_array('date')        
        dates = dates.getInfo()
        
        ndvi = modis.select('NDVI');
        ndvi_array = ndvi.toArray();
        max_ndvi_date = ndvi_array.arrayArgmax();
        max_ndvi_image = ee.Image(max_ndvi_date).arrayProject([0]).arrayFlatten([['maxDate_start', 'band2']]).clip(aoi).select("maxDate_start");

        # Remap values so '0' doesnt overlap with nodata        
        fromValues = []
        i = 0
        for date in dates:
            fromValues.append(i)
            i +=1

        toValues = []
        for val in fromValues:
            toValues.append(val+1)

        max_ndvi_image = max_ndvi_image.remap(**{
          "from": fromValues,
          "to": toValues,
          "defaultValue": 0,
          "bandName": 'maxDate_start'
        });
        
        
        # Create a value - date hashmap for future remapping
        self.date_dict = {}
        i = 1
        for date in dates:
            self.date_dict[str(i)] = date
            i+=1
        self.date_dict["0"] = -99
        print(f"Done with MODIS best date calculation.. - {time.time()-start} sec")
        
        # Download modis tiles at parent resolution
        DATA_DIR = self.data_dir
        TILE_DIR = f"{self.data_dir}/interim/tiles"
        Path(TILE_DIR).mkdir(parents=True, exist_ok=True)
        
        parent = gpd.read_file(f"{DATA_DIR}/interim/parent.gpkg")
        for index,row in parent.iterrows():
            aoi = ee.Geometry.Rectangle(row.geometry.bounds)
            geemap.ee_export_image(
                max_ndvi_image, 
                filename=f"{TILE_DIR}/{row.pgrid_id}.tif", 
                scale=250, 
                region=aoi, 
                file_per_band=False
            )
        
        
        print(f"Downloaded tiles.. - {time.time()-start} sec")

        
        
        if os.path.exists(f"{DATA_DIR}/interim/temp.vrt"):
            os.remove(f"{DATA_DIR}/interim/temp.vrt")
        if os.path.exists(f"{DATA_DIR}/interim/merged.tif"):
            os.remove(f"{DATA_DIR}/interim/merged.tif")
        if os.path.exists(f"{DATA_DIR}/interim/merged.gpkg"):
            os.remove(f"{DATA_DIR}/interim/merged.gpkg")
        if os.path.exists(f"{DATA_DIR}/interim/shell.tif"):
            os.remove(f"{DATA_DIR}/interim/shell.tif")
        if os.path.exists(f"{DATA_DIR}/interim/shell.tif"):
            os.remove(f"{DATA_DIR}/interim/shell.gpkg")
        if os.path.exists(f"{DATA_DIR}/interim/centroids.gpkg"):
            os.remove(f"{DATA_DIR}/interim/centroids.gpkg")

        # Merge downloaded modis tiles into one
        os.system(f'gdalbuildvrt {DATA_DIR}/interim/temp.vrt {DATA_DIR}/interim/tiles/*.tif -srcnodata "0"')
        os.system(f'gdal_merge.py -o {DATA_DIR}/interim/merged.tif {DATA_DIR}/interim/temp.vrt')
        
        print(f"Merged tiles.. - {time.time()-start} sec")
        
        
        # Create shell GDF (workaround because polygonize doesn't uncombine tiles with same value)
        with rio.open(f"{DATA_DIR}/interim/merged.tif") as src:
            array = src.read(1)
            transform = src.transform
            crs = src.crs
            profile = src.profile

        h, w = array.shape

        new_array = np.arange(h*w).reshape(h,w)

        with rio.Env():

            # Write an array as a raster band to a new 8-bit file. For
            # the new file's profile, we start with the profile of the source
            profile = src.profile

            # And then change the band count to 1, set the
            # dtype to uint8, and specify LZW compression.
            profile.update(
                dtype=rio.uint32,
                count=1,
                compress='lzw')

            with rio.open(f'{DATA_DIR}/interim/shell.tif', 'w', **profile) as dst:
                dst.write(new_array.astype(rio.uint32), 1)
        
        print(f"Shell GDF created.. - {time.time()-start} sec")
        
    
        os.system(f'gdal_polygonize.py {DATA_DIR}/interim/shell.tif -b 1 -f "GPKG" {DATA_DIR}/interim/shell.gpkg OUTPUT DateCode')
        print(f"Polygonized shell.. - {time.time()-start} sec")
        
        os.system(f'gdal_polygonize.py {DATA_DIR}/interim/merged.tif -b 1 -f "GPKG" {DATA_DIR}/interim/merged.gpkg OUTPUT DateCode')
        print(f"Polygonized merged.. - {time.time()-start} sec")
        
        with rio.open(f"{DATA_DIR}/interim/merged.tif") as src:
            band1 = src.read(1)
            height = band1.shape[0]
            width = band1.shape[1]
            cols, rows = np.meshgrid(np.arange(width), np.arange(height))
            xs, ys = rio.transform.xy(src.transform, rows, cols)
            lons = np.array(xs)
            lats = np.array(ys)

            points = gpd.GeoSeries(
                list(zip(lons.flatten(), lats.flatten()))).map(Point)

            # use the feature loop in case shp is multipolygon
            geoms = points.values
            features = [i for i in range(len(geoms))]

            out = gpd.GeoDataFrame(
                {'feature': features, 'geometry': geoms}, crs=src.crs)
            out.to_file(f"{DATA_DIR}/interim/centroids.gpkg", driver="GPKG")
        merged = gpd.read_file(f"{DATA_DIR}/interim/merged.gpkg")
        centroids = gpd.read_file(f"{DATA_DIR}/interim/centroids.gpkg")
        centroids = centroids.sjoin(merged, how="inner", predicate='intersects')
        centroids = centroids[['feature', 'geometry', 'DateCode']]
        print(f"Created Centroids.. - {time.time()-start} sec")
        
        
        
        shell = gpd.read_file(f"{DATA_DIR}/interim/shell.gpkg")
        child = shell.sjoin(centroids,  how="inner", predicate='intersects')
        child = child[["DateCode_right", "geometry"]]
        child['DateCode_right'] = child['DateCode_right'].astype(str)
        child = child.replace({"DateCode_right": self.date_dict})
        child = child[child['DateCode_right'] != "-99"]
        child = child.reset_index()
        child.columns = ["GRID_ID", "BSD", "geometry"]
        child.to_file(f"{DATA_DIR}/interim/child.gpkg", driver="GPKG")
        child = gpd.read_file(f"{DATA_DIR}/interim/child.gpkg")
#         print(child.columns)
        child = child[child['BSD'] != "-99"]
        child = child.reset_index()
        child = child[['index', 'BSD', 'geometry']]
        child.columns = ['grid_id', 'BSD', 'geometry']
        child.to_file(f"{DATA_DIR}/interim/child.gpkg", driver="GPKG")
        print(f"Saved child GDF. Completed! - {time.time()-start} sec")
        
        

In [18]:
dh = DatesHelper("../../data", "../../data/inputs/aoi.gpkg", ['2019-01-01', '2019-06-15'])

In [19]:
dh.extract_best_dates()

Done with MODIS best date calculation.. - 0.15477609634399414 sec
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/0.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/1.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/2.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/3.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/4.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/5.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/6.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/7.tif
Genera

Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/27.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/28.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/29.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/30.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/31.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/32.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/33.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/34.tif
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/int

<ipython-input-17-5a9038e08d7f>:152: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  points = gpd.GeoSeries(
/home/arogya/.local/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Created Centroids.. - 67.31403374671936 sec


/home/arogya/.local/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/arogya/.local/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Saved child GDF. Completed! - 111.81511664390564 sec


In [20]:
import geopandas as gpd

In [21]:
child = gpd.read_file("../../data/interim/child.gpkg")

In [22]:
child

,index,BSD,geometry
0,4122,2019-03-06,"POLYGON ((64.03416 31.53985, 64.03416 31.53760..."
1,4123,2019-03-06,"POLYGON ((64.03416 31.53985, 64.03416 31.53760..."
2,4124,2019-03-06,"POLYGON ((64.03416 31.53985, 64.03416 31.53760..."
3,4125,2019-03-06,"POLYGON ((64.03416 31.53985, 64.03416 31.53760..."
4,4126,2019-03-06,"POLYGON ((64.03416 31.53985, 64.03416 31.53760..."
...,...,...,...
80509,99751,2019-04-23,"POLYGON ((64.17340 31.38264, 64.17340 31.38040..."
80510,99752,2019-04-23,"POLYGON ((64.17340 31.38264, 64.17340 31.38040..."
80511,99753,2019-04-23,"POLYGON ((64.17340 31.38264, 64.17340 31.38040..."
80512,99754,2019-04-23,"POLYGON ((64.17340 31.38264, 64.17340 31.38040..."


In [34]:
a=ee.List(['fafa', 'fofo'])

In [35]:
a.getInfo()

['fafa', 'fofo']

In [22]:
import eeconvert as eeconvert
import ee
import geemap
import geopandas as gpd
import rasterio as rio
import numpy as np
import rasterio.features as features

In [23]:
ee.Initialize()

In [24]:
afghanistan = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0").filter("ADM0_NAME == 'Afghanistan'");
modis = ee.ImageCollection('MODIS/061/MOD13Q1').filter(ee.Filter.date('2019-01-01', '2019-06-15'));
dates = modis.map(lambda x: ee.Feature(None, {'date': x.date().format('YYYY-MM-dd')})).distinct('date').aggregate_array('date')

In [25]:
ndvi = modis.select('NDVI');
# ndvi1 = ndvi.max();
ndvi_Array = ndvi.toArray();
test = ndvi_Array.arrayArgmax();
test1 = ee.Image(test).arrayProject([0]).arrayFlatten([['maxDate_start', 'band2']]).clip(afghanistan).select("maxDate_start");
projection = test1.projection()
reduction = test1.reduceRegion(ee.Reducer.frequencyHistogram(), afghanistan, 250, projection.crs)
# values = ee.Dictionary(reduction.get(test1.bandNames)).keys().map(ee.Number.parse);

fromValues = []
i = 0
for date in dates.getInfo():
    fromValues.append(i)
    i +=1

toValues = []
for val in fromValues:
    toValues.append(val+1)

test1 = test1.remap(**{
  "from": fromValues,
  "to": toValues,
  "defaultValue": 0,
  "bandName": 'maxDate_start'
});

In [26]:
# parent = gpd.read_file("/data/tmp/arogya/data/interim/parent.gpkg")
parent = gpd.read_file("../../data/interim/parent.gpkg")

for index,row in parent.iterrows():
    print(row.pgrid_id)
    aoi = ee.Geometry.Rectangle(row.geometry.bounds)
    geemap.ee_export_image(
        test1, 
        filename=f"../../data/interim/tiles/{row.pgrid_id}.tif", 
        scale=250, 
        region=aoi, 
        file_per_band=False
    )

# ee.Geometry.Rectangle([64.0321287595154, 31.3779740304129, 64.17798786794364, 31.542232512963988])

0
Generating URL ...
Please wait ...
Data downloaded to /home/arogya/projects/afg-clustering/data/interim/tiles/0.tif


In [27]:
import os
os.system(f'gdalbuildvrt ../../data/interim/temp.vrt ../../data/interim/tiles/*.tif -srcnodata "0"')
# print(f"--- Merging for District {dist_id}: {time.time() - self.start_time} seconds ---")
os.system(f'gdal_merge.py -o ../../data/interim/merged.tif ../../data/interim/temp.vrt')



0

In [28]:
with rio.open("../../data/interim/merged.tif") as src:
    array = src.read(1)
    transform = src.transform
    crs = src.crs
    profile = src.profile
    
h, w = array.shape

new_array = np.arange(h*w).reshape(h,w)

with rio.Env():

    # Write an array as a raster band to a new 8-bit file. For
    # the new file's profile, we start with the profile of the source
    profile = src.profile

    # And then change the band count to 1, set the
    # dtype to uint8, and specify LZW compression.
    profile.update(
        dtype=rio.uint32,
        count=1,
        compress='lzw')

    with rio.open('../../data/interim/shell.tif', 'w', **profile) as dst:
        dst.write(new_array.astype(rio.uint32), 1)

In [29]:
os.system('gdal_polygonize.py ../../data/interim/shell.tif -b 1 -f "GPKG" ../../data/interim/shell.gpkg OUTPUT DateCode')

0

In [30]:
os.system('gdal_polygonize.py ../../data/interim/merged.tif -b 1 -f "GPKG" ../../data/interim/merged.gpkg OUTPUT DateCode')

0

In [31]:
shell = gpd.read_file("../../data/interim/shell.gpkg")
merged = gpd.read_file("../../data/interim/merged.gpkg")

In [32]:
with_dates = shell.sjoin(merged,  how="inner", predicate='intersects')
with_dates.to_file("../../data/interim/with_dates.gpkg")

/home/arogya/.local/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [33]:
import rasterio
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

path = '../../data/interim/merged.tif'


with rasterio.open(path) as src:
    band1 = src.read(1)
    height = band1.shape[0]
    width = band1.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(src.transform, rows, cols)
    lons = np.array(xs)
    lats = np.array(ys)

    points = gpd.GeoSeries(
        list(zip(lons.flatten(), lats.flatten()))).map(Point)

    # use the feature loop in case shp is multipolygon
    geoms = points.values
    features = [i for i in range(len(geoms))]

    out = gpd.GeoDataFrame(
        {'feature': features, 'geometry': geoms}, crs=src.crs)
    out.to_file("../../data/interim/centroids.gpkg", driver="GPKG")

<ipython-input-33-d53ca80e6892>:18: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  points = gpd.GeoSeries(
/home/arogya/.local/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [41]:
centroids = gpd.read_file("../../data/interim/centroids.gpkg")
centroids = centroids.sjoin(merged, how="inner", predicate='intersects')
centroids = centroids[['feature', 'geometry', 'DateCode']]
centroids

,feature,geometry,DateCode
0,0,POINT (64.02630 31.54996),6
3,3,POINT (64.03304 31.54996),6
4,4,POINT (64.03528 31.54996),6
5,5,POINT (64.03753 31.54996),6
123,123,POINT (64.02630 31.54771),6
...,...,...,...
13030,13030,POINT (64.28456 31.31415),6
13034,13034,POINT (64.29355 31.31415),6
13035,13035,POINT (64.29579 31.31415),4
13036,13036,POINT (64.29804 31.31415),4


In [43]:
shell.sjoin(centroids,  how="inner", predicate='intersects').to_file("../../data/interim/centroid_dates.gpkg", driver="GPKG")

/home/arogya/.local/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [35]:
import rasterio.features
maskShape = rasterio.features.shapes(mask.astype('uint8'))
mypoly=[]
for vec in maskShape:
    mypoly.append(vec[0])
print(mypoly)


NameError: name 'mask' is not defined

In [ ]:
src = rio.open("../../data/interim/tiles/merged.tif")
input_array = src.read(1)
input_transform = src.transform
input_crs = src.crs



# Create array with a unique value per cell
unique_pixels = np.arange(input_array.size).reshape(input_array.shape)

# Vectorise each unique feature in array
vectors = features.shapes(
    source=unique_pixels.astype(np.int16), transform=input_transform
)

# Extract polygons and values from generator
vectors = list(vectors)
values = [value for polygon, value in vectors]
polygons = [shape(polygon) for polygon, value in vectors]

# Create a geopandas dataframe populated with the polygon shapes
poly_gdf = gpd.GeoDataFrame(data={"id": values}, geometry=polygons, crs=input_crs)


In [ ]:
from osgeo import gdal, ogr, osr

in_path = '../../data/interim/tiles/merged.tif'

out_path = '../../data/interim/tiles/merged.gpkg'

#  get raster datasource
src_ds = gdal.Open( in_path )
#
srcband = src_ds.GetRasterBand(1)
dst_layername = 'MAXNDVI'
drv = ogr.GetDriverByName("GPKG")
dst_ds = drv.CreateDataSource( out_path )

sp_ref = osr.SpatialReference()
sp_ref.SetFromUserInput('EPSG:4326')

dst_layer = dst_ds.CreateLayer(dst_layername, srs = sp_ref )

fld = ogr.FieldDefn("HA", ogr.OFTInteger)
dst_layer.CreateField(fld)
dst_field = dst_layer.GetLayerDefn().GetFieldIndex("HA")

gdal.Polygonize(srcband, None, dst_layer, dst_field, [], callback=None)

del src_ds
del dst_ds

In [ ]:
geemap.ee_export_image(
    test1, filename="../../data/interim/modis.tif", scale=250, region=ee.Geometry.Rectangle([64.0321287595154, 31.3779740304129, 64.17798786794364, 31.542232512963988]), file_per_band=False
)

In [ ]:
import rasterio as rio 
import geopandas as gpd
import pandas as pd
import numpy as np
import rioxarray
import matplotlib.pyplot as plt
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

In [ ]:
gdf0 = gpd.read_file("../../data/interim/child.gpkg")
gdf0.unary_union.bounds

In [ ]:
# from rasterstats import zonal_stats

with rio.open("/data/tmp/arogya/data/inputs/all_afg_raster.tiff", nodata=0) as src:
    affine = src.transform
    array = src.read(1)
    
#     df_zonal_stats = pd.DataFrame(zonal_stats(gdf, array, affine=affine))

# # # adding statistics back to original GeoDataFrame
# gdf2 = pd.concat([gdf, df_zonal_stats], axis=1) 

In [ ]:
rds = rioxarray.open_rasterio("/data/tmp/arogya/data/inputs/all_afg_raster.tiff")
rds.name = "data"
df = rds.squeeze().to_dataframe().reset_index()
geometry = gpd.points_from_xy(df.x, df.y)
gdf = gpd.GeoDataFrame(df, crs=rds.rio.crs, geometry=geometry)

In [ ]:
a = gdf.sjoin(gdf0, how="inner", predicate='intersects')
a

In [ ]:
fig, ax = plt.subplots(1,1, dpi=200, figsize=(30, 60))
a.plot(ax=ax)
gdf0.boundary.plot(ax=ax)
# ax.imshow(array, cmap='jet')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))


rst = rasterio.open("/data/tmp/arogya/data/inputs/all_afg_raster.tiff")
red = rst.read(1)
rio.plot.show(red, ax=ax)

In [ ]:
import itertools
import rasterio
from shapely.geometry import box
import geopandas as gpd

with rasterio.open("../../data/interim/tiles/merged.tif") as dataset:
    data = dataset.read(1)

    t = dataset.transform

    move_x = t[0]
    # t[4] is negative, as raster start upper left 0,0 and goes down
    # later for steps calculation (ymin=...) we use plus instead of minus
    move_y = t[4]

    height = dataset.height
    width = dataset.width 

    polygons = []
    indices = list(itertools.product(range(width), range(height)))
    for x,y in indices:
        x_min, y_max = t * (x,y)
        x_max = x_min + move_x
        y_min = y_max + move_y
        polygons.append(box(x_min, y_min, x_max, y_max))

data_list = []
for x,y in indices:
    data_list.append(data[y,x])
    
gdf = gpd.GeoDataFrame(data=data_list, crs={'init':'epsg:4236'}, geometry=polygons, columns=['value'])
gdf.to_file("../../data/interim/merged2.gpkg", driver="GPKG")